In [ ]:
#Source: https://github.com/bepoetree/MTTOD

!git clone https://github.com/vonCount/MTTOD.git && cd MTTOD/ && git lfs install && git lfs pull

!ls -l MTTOD/

Cloning into 'MTTOD'...
remote: Enumerating objects: 85, done.
remote: Counting objects: 100% (85/85), done.
remote: Compressing objects: 100% (62/62), done.
remote: Total 85 (delta 42), reused 46 (delta 19), pack-reused 0
Unpacking objects: 100% (85/85), done.
Updated Git hooks.
Git LFS initialized.
total 34668
-rw-r--r-- 1 jovyan jovyan    11350 May 30 10:33 LICENSE
-rw-r--r-- 1 jovyan jovyan      635 May 30 10:33 NOTICE
-rw-r--r-- 1 jovyan jovyan     3360 May 30 10:33 README.md
-rw-r--r-- 1 jovyan jovyan     6107 May 30 10:33 config.py
-rwxr-xr-x 1 jovyan jovyan     2067 May 30 10:33 convert.py
-rwxr-xr-x 1 jovyan jovyan 35136254 May 30 10:33 data.zip
-rwxr-xr-x 1 jovyan jovyan    27602 May 30 10:33 evaluator.py
-rw-r--r-- 1 jovyan jovyan     9560 May 30 10:33 external_knowledges.py
-rw-r--r-- 1 jovyan jovyan     1615 May 30 10:33 main.py
-rw-r--r-- 1 jovyan jovyan     8692 May 30 10:33 model.py
-rw-r--r-- 1 jovyan jovyan    27432 May 30 10:33 preprocess.py
-rwxr-xr-x 1 jovyan jovya

In [ ]:
# multiwoz data is packed inside repo in data.zip
!unzip MTTOD/data.zip -d ./data

Archive:  MTTOD/data.zip
  inflating: ./data/MultiWOZ_2.0/db/bus_db.json  
  inflating: ./data/MultiWOZ_2.0/db/value_set.json  
  inflating: ./data/MultiWOZ_2.0/db/hospital_db.json  
  inflating: ./data/MultiWOZ_2.0/db/restaurant_db.json  
  inflating: ./data/MultiWOZ_2.0/db/train_db.json  
  inflating: ./data/MultiWOZ_2.0/db/attraction_db.json  
  inflating: ./data/MultiWOZ_2.0/db/hotel_db.json  
  inflating: ./data/MultiWOZ_2.0/db/police_db.json  
  inflating: ./data/MultiWOZ_2.0/db/taxi_db.json  
  inflating: ./data/MultiWOZ_2.0/ontology.json  
  inflating: ./data/MultiWOZ_2.0/testListFile.json  
  inflating: ./data/MultiWOZ_2.0/annotated_user_da_with_span_full.json  
  inflating: ./data/MultiWOZ_2.0/valListFile.json  
  inflating: ./data/MultiWOZ_2.0/dial_by_domain.json  
  inflating: ./data/MultiWOZ_2.0/slot_descriptions.json  
  inflating: ./data/MultiWOZ_2.1/db/value_set.json  
  inflating: ./data/MultiWOZ_2.1/db/hospital_db.json  
  inflating: ./data/MultiWOZ_2.1/db/restaurant_

In [ ]:
!pip install -r MTTOD/requirements.txt
!python -m spacy download en_core_web_sm

  Using cached torch-1.7.1-cp37-cp37m-manylinux1_x86_64.whl (776.8 MB)
  Attempting uninstall: torch
    Found existing installation: torch 1.0.1.post2
    Uninstalling torch-1.0.1.post2:
      Successfully uninstalled torch-1.0.1.post2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.11.2+cu111 requires torch==1.10.1, but you have torch 1.7.1 which is incompatible.
torchaudio 0.10.1+rocm4.1 requires torch==1.10.1, but you have torch 1.7.1 which is incompatible.
pykeen 1.8.1 requires torch>=1.10; platform_system != "Windows", but you have torch 1.7.1 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 55.9 MB/s eta 0:00:0000:0100:01
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [ ]:
import os
os.environ['VERSION'] = '2.1'
os.environ['MODEL_DIR'] = './MTTOD/models/'
os.environ['CHECKPOINT'] = './MTTOD/models/ckpt-epoch1'
os.environ['MODEL_OUTPUT'] = 'preds.json'
os.environ['BATCH_SIZE'] = '4'

In [ ]:
!python MTTOD/preprocess.py -version=$VERSION

DB value set processed !
[attraction] DB processed !
[hotel] DB processed !
[restaurant] DB processed !
[taxi] DB processed !
[train] DB processed !
[hospital] DB processed !
[police] DB processed !
Processing entity values in [attraction]
Processing entity values in [hotel]
Processing entity values in [restaurant]
Processing entity values in [taxi]
Processing entity values in [train]
Processing entity values in [hospital]
Processing entity values in [police]
single delex value dict saved!
multi delex value dict saved!
ambiguous value dict saved!
100%|█████████████████████████████████████| 10438/10438 [15:14<00:00, 11.42it/s]
Save preprocessed data to data/MultiWOZ_2.1/processed (#train: 8434, #dev: 999, #test: 1000)


In [ ]:
# pip install torch==1.10.1+cu111 torchvision==0.11.2+cu111 torchaudio==0.10.1 -f https://download.pytorch.org/whl/torch_stable.html

In [ ]:
!mkdir MTTOD/models

In [ ]:
#Training

!python MTTOD/main.py -version $VERSION -run_type train -model_dir $MODEL_DIR -batch_size=32 -epochs=1

05/30/2022 12:41:09  [INFO] Device: cuda (the number of GPUs: 1)
05/30/2022 12:41:09  [INFO] Set random seed to 42
05/30/2022 12:41:12  [INFO] Load encoded data from data/MultiWOZ_2.1/processed/encoded_data.pkl
05/30/2022 12:41:13  [INFO] Load model from t5-base
Some weights of T5WithSpan were not initialized from the model checkpoint at t5-base and are newly initialized: ['resp_decoder.block.0.layer.0.SelfAttention.q.weight', 'resp_decoder.block.0.layer.0.SelfAttention.k.weight', 'resp_decoder.block.0.layer.0.SelfAttention.v.weight', 'resp_decoder.block.0.layer.0.SelfAttention.o.weight', 'resp_decoder.block.0.layer.0.SelfAttention.relative_attention_bias.weight', 'resp_decoder.block.0.layer.0.layer_norm.weight', 'resp_decoder.block.0.layer.1.EncDecAttention.q.weight', 'resp_decoder.block.0.layer.1.EncDecAttention.k.weight', 'resp_decoder.block.0.layer.1.EncDecAttention.v.weight', 'resp_decoder.block.0.layer.1.EncDecAttention.o.weight', 'resp_decoder.block.0.layer.1.layer_norm.weight',

In [ ]:
#Evaluation

!python MTTOD/main.py -run_type predict -ckpt $CHECKPOINT -output $MODEL_OUTPUT -batch_size $BATCH_SIZE

05/30/2022 13:36:27  [INFO] Device: cuda (the number of GPUs: 1)
05/30/2022 13:36:27  [INFO] Set random seed to 42
05/30/2022 13:36:30  [INFO] Load encoded data from data/MultiWOZ_2.1/processed/encoded_data.pkl
05/30/2022 13:36:32  [INFO] Load model from ./MTTOD/models/ckpt-epoch1
Prediction: 100%|█████████████████████████████| 250/250 [24:36<00:00,  5.91s/it]
05/30/2022 14:01:27  [INFO] match: 74.70; success: 67.10; bleu: 13.98; score: 84.88
